#### ML HW2 Xinmeng Zhang

In [4]:
# library for dataframes
import pandas as pd

# scientific computing libraries
import numpy as np
import numpy.random as rn
import scipy.stats as st
from collections import Counter

# plotting libraries
import matplotlib.pyplot as plt

import seaborn as sns

### Question 4 MNIST classification

In [28]:
# Load the data and labels and normalize it
mnist = np.load('MNIST_data.npy') / 255.0
mnist_labels = np.load('MNIST_labels.npy')

from sklearn.model_selection import train_test_split

# Assuming you have your data stored in numpy arrays X and Y
# Split the data into 80% training and 20% test
mnist_train, mnist_test, train_labels, test_labels= train_test_split(mnist, mnist_labels, test_size=0.2, random_state=42)

# Further split the training data into 60% training and 20% development
mnist_train, mnist_dev, train_labels, dev_labels = train_test_split(mnist_train, train_labels, test_size=0.25, random_state=42)

# mnist_train, train_labels: Training data (60% of the original data)
# mnist_dev, dev_labels: Development data (20% of the original data)
# mnist_test, test_labels: Test data (20% of the original data)


- <span style="font-size: x-large;">Bayes rule with Bernoulli mixtures</span>

Read in the MNIST data set and binarize as in HW3. Estimate a separate mixture
model for each class with $M = 1,5,10,20$. So in total you will have $M \cdot 10$ models.
For M = 1 you don’t need the EM algorithm. Use the development set to choose
the optimal M and then retrain the mixtures with the training and development sets.
Classify the test set data using Bayes’ rule with your estimated class mixture models.
You can assume that $\pi_c = 1/10, c = 1,...,10.$

In [6]:
# Binarize the data
train_X_binarized = (mnist_train > 0.5).astype(int)
dev_X_binarized = (mnist_dev > 0.5).astype(int)
test_X_binarized = (mnist_test > 0.5).astype(int)

In [56]:
def EM_algorithm_binarized(X, num_components, num_iterations=100):
    """
    Runs the EM algorithm on binarized data X to fit a mixture of Bernoulli distributions.
    Returns the learned parameters pi, mu, and w.
    """
    num_samples, num_features = X.shape
    
    # Initialize pi and w matrix randomly
    pi = np.random.rand(num_components)
    pi /= pi.sum()
    w = np.random.rand(num_components, num_features)
    w /= w.sum

    # Run EM algorithm
    for i in range(num_iterations):
        # E-step: compute responsibilities
        z = X.dot(w.T)
        z *= pi
        
        # Normalize responsibilities
        z /= z.sum
        
        # M-step: update parameters
        w = z.T.dot(X) / z.sum

        mu = (w + 1) / 2  # Avoid probabilities of 0 or 1
        pi = z.mean(axis=0)
        
    # Return learned parameters
    return pi, mu, w

In [9]:
# Train mixture models for each class with M components
M_values = [1, 5, 10, 20]  # M values to use
pi_dict = {}  # Dictionary to store learned pi values for each class and M
mu_dict = {}  # Dictionary to store learned mu values for each class and M

for M in M_values:
    for c in range(10):
        if M == 1:
            # For M=1, use a uniform distribution as the mixture model
            pi = np.ones(1)
            mu = np.random.rand(1)
        else:
            # Extract data for current class
            class_indices = np.where(train_labels == str(c))[0]
            if len(class_indices) == 0:
                # Skip if there are no samples for current class
                continue
            class_X = train_X_binarized[class_indices]
            
            # Train mixture model for current class with M components
            pi, mu, w = EM_algorithm_binarized(class_X, num_components=M)
        
        # Store learned parameters for current class and M
        pi_dict[(c, M)] = pi
        mu_dict[(c, M)] = mu

d:\anaconda3\envs\cse160\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [45]:
#mu_dict.keys()
pi_dict.values()

dict_values([array([1.]), array([1.]), array([1.]), array([1.]), array([1.]), array([1.]), array([1.]), array([1.]), array([1.]), array([1.])])

In [ ]:
# Initialize dictionary to store mixture models for different M values for each class
mixture_models = {}

# Loop over each class
for c in range(10):
    print("Fitting class", c)
    X_train_c = np.where(train_labels == str(c))
    X_dev_c = np.where(dev_labels == str(c))
    
    # Initialize dictionary to store mixture models for this class
    mixture_models_c = {}
    
    # Loop over each M value
    for M in M_values:
        if M == 1:
            # If M is 1, use a single Bernoulli distribution (no need for EM)
            pi, mu, w = EM_algorithm_binarized(X_train_c, 1, num_iterations=0)
        else:
            # Otherwise, use EM to estimate mixture of Bernoulli distributions
            pi, mu, w = EM_algorithm_binarized(X_train_c, M)
        
        # Store the trained mixture model for this M value
        mixture_models_c[M] = (pi, mu, w)
    
    # Choose the optimal M value based on the development set
    best_M = None
    best_score = None
    for M in M_values:
        pi, mu, w = mixture_models_c[M]
        dev_score = np.sum(X_dev_c.dot(np.log(mu).T) + (1 - X_dev_c).dot(np.log(1 - mu).T) + np.log(pi), axis=1)
        dev_score = dev_score.mean()
        if best_score is None or dev_score > best_score:
            best_M = M
            best_score = dev_score

In [ ]:
pi,mu,w = EM_algorithm_binarized(mnist_test, num_components=5)
num_test_samples = mnist_test.shape[0]
log_probs = np.dot(mnist_test, np.log(w.T)) + np.dot(1 - mnist_test, np.log(1 - w.T))
log_probs += np.log(pi)
predicted_labels = np.argmax(log_probs, axis=1) + 1  # Add 1 to convert from zero-indexed to one-indexed labels

- <span style="font-size: x-large;">Logistic Regression</span>
1. Raw data

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lg=LogisticRegression(fit_intercept=True, C=100000, penalty='l2', multi_class='multinomial',solver='lbfgs')

# Train the model on the training set
lg.fit(mnist_train, train_labels)

# Evaluate the model on the development set
dev_predictions = lg.predict(mnist_dev)
dev_accuracy = accuracy_score(dev_labels, dev_predictions)
print("Accuracy on development set:", dev_accuracy)

# Search for the optimal value of C on the development set
best_c = None
best_accuracy = 0.0
c_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]
for c in c_values:
    lg = LogisticRegression(fit_intercept=True, C=c, penalty='l2', multi_class='multinomial', solver='lbfgs')
    lg.fit(mnist_train, train_labels)
    dev_predictions = lg.predict(mnist_dev)
    dev_accuracy = accuracy_score(dev_labels, dev_predictions)
    print("C =", c, "Accuracy on development set:", dev_accuracy)
    if dev_accuracy > best_accuracy:
        best_accuracy = dev_accuracy
        best_c = c

# Retrain the model with the optimal value of C using training and development set combined
lg = LogisticRegression(fit_intercept=True, C=best_c, penalty='l2', multi_class='multinomial', solver='lbfgs')
lg.fit(np.concatenate((mnist_train, mnist_dev)), np.concatenate((train_labels, dev_labels)))

# Evaluate the model on the test set
test_predictions = lg.predict(mnist_test)
test_accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy on test set:", test_accuracy)


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy on development set: 0.9185
C = 1e-05 Accuracy on development set: 0.7122142857142857
C = 0.0001 Accuracy on development set: 0.8388571428571429


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


C = 0.001 Accuracy on development set: 0.8872857142857142


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


...

d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


C = 10000.0 Accuracy on development set: 0.9187857142857143


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


C = 100000.0 Accuracy on development set: 0.9185
Accuracy on test set: 0.9217857142857143


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


2. PCA

In [ ]:
# PCA function
def PCA(X, k):
    # center data
    X_centered = X - np.mean(X, axis=0)
    
    # compute covariance matrix
    cov_matrix = np.cov(X_centered.T)
    
    # compute eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    # sort eigenvectors by eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvectors = eigenvectors[:,idx]
    
    # return first k eigenvectors
    return eigenvectors[:,:k], eigenvalues


In [38]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Perform PCA on the training set to obtain the principal components
pca = PCA(n_components=50)
pca.fit(mnist_train)
train_pcs = pca.transform(mnist_train)

# Perform PCA on the development set to obtain the principal components
dev_pcs = pca.transform(mnist_dev)

# Perform PCA on the test set to obtain the principal components
test_pcs = pca.transform(mnist_test)

# Search for the optimal number of principal components (k) and optimal value of C on the development set
best_k = None
best_c = None
best_accuracy = 0.0
k_values = [1, 5, 10, 50, 100, 200]
c_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0]
for k in k_values:
    # Train logistic regression model with k principal components for each value of C
    for c in c_values:
        lg = LogisticRegression(fit_intercept=True, C=c, penalty='l2', multi_class='multinomial', solver='lbfgs')
        lg.fit(train_pcs[:, :k], train_labels)
        dev_predictions = lg.predict(dev_pcs[:, :k])
        dev_accuracy = accuracy_score(dev_labels, dev_predictions)
        print("k =", k, "C =", c, "Accuracy on development set:", dev_accuracy)
        if dev_accuracy > best_accuracy:
            best_accuracy = dev_accuracy
            best_k = k
            best_c = c

# Retrain the model with the optimal number of principal components and optimal value of C
lg = LogisticRegression(fit_intercept=True, C=best_c, penalty='l2', multi_class='multinomial', solver='lbfgs')
lg.fit(np.concatenate((train_pcs[:, :best_k], dev_pcs[:, :best_k])), np.concatenate((train_labels, dev_labels)))

# Evaluate the model on the test set
test_predictions = lg.predict(test_pcs[:, :best_k])
test_accuracy = accuracy_score(test_labels, test_predictions)
print("Optimal k:", best_k)
print("Optimal C:", best_c)
print("Accuracy on test set:", test_accuracy)


k = 1 C = 1e-05 Accuracy on development set: 0.21614285714285714
k = 1 C = 0.0001 Accuracy on development set: 0.27214285714285713
k = 1 C = 0.001 Accuracy on development set: 0.3048571428571429
k = 1 C = 0.01 Accuracy on development set: 0.3105
k = 1 C = 0.1 Accuracy on development set: 0.3107857142857143
k = 1 C = 1.0 Accuracy on development set: 0.3107142857142857
k = 1 C = 10.0 Accuracy on development set: 0.3107857142857143
k = 1 C = 100.0 Accuracy on development set: 0.3107857142857143
k = 1 C = 1000.0 Accuracy on development set: 0.3107857142857143
k = 1 C = 10000.0 Accuracy on development set: 0.3107857142857143
k = 1 C = 100000.0 Accuracy on development set: 0.3107857142857143
k = 5 C = 1e-05 Accuracy on development set: 0.5146428571428572
k = 5 C = 0.0001 Accuracy on development set: 0.6122142857142857
k = 5 C = 0.001 Accuracy on development set: 0.6565714285714286
k = 5 C = 0.01 Accuracy on development set: 0.6680714285714285
k = 5 C = 0.1 Accuracy on development set: 0.6699

d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


k = 50 C = 0.1 Accuracy on development set: 0.9055


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


k = 50 C = 1.0 Accuracy on development set: 0.9057142857142857


...

d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


k = 200 C = 10000.0 Accuracy on development set: 0.9057142857142857


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


k = 200 C = 100000.0 Accuracy on development set: 0.9057142857142857
Optimal k: 50
Optimal C: 10.0
Accuracy on test set: 0.9062857142857143


d:\anaconda3\envs\cse160\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


- <span style="font-size: x-large;">Stochastic gradient descent</span>


In [ ]:
import numpy as np
from scipy.special import softmax, logsumexp
import matplotlib.pyplot as plt

def sgd(X, y, X_val, y_val, reg_param, learning_rate, batch_size, num_epochs):
    """
    Implements stochastic gradient descent (SGD) for logistic regression with L2 regularization.

    Args:
    X: array, shape (num_train, num_features), training data.
    y: array, shape (num_train,), training labels.
    X_val: array, shape (num_val, num_features), validation data.
    y_val: array, shape (num_val,), validation labels.
    reg_param: float, regularization parameter.
    learning_rate: float, learning rate.
    batch_size: int, mini-batch size.
    num_epochs: int, number of training epochs.

    Returns:
    W: array, shape (num_features, num_classes), learned weights.
    b: array, shape (num_classes,), learned biases.
    train_losses: list, training losses for each epoch.
    val_losses: list, validation losses for each epoch.
    train_error_rates: list, training error rates for each epoch.
    val_error_rates: list, validation error rates for each epoch.
    """

    num_train, num_features = X.shape
    num_val = X_val.shape[0]
    num_classes = np.max(y) + 1

    # Initialize model parameters
    W = np.zeros((num_features, num_classes))
    b = np.zeros(num_classes)

    # Initialize lists to store losses and error rates for each epoch
    train_losses = []
    val_losses = []
    train_error_rates = []
    val_error_rates = []

    # Perform SGD
    for epoch in range(num_epochs):
        # Shuffle the training data for each epoch
        permutation = np.random.permutation(num_train)
        X = X[permutation]
        y = y[permutation]

        # Process mini-batches
        for i in range(0, num_train, batch_size):
            # Get mini-batch
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]

            # Compute scores
            scores = np.dot(X_batch, W) + b

            # Compute softmax probabilities
            probs = softmax(scores, axis=1)

            # Compute negative log-likelihood loss with L2 regularization
            loss = -np.sum(np.log(probs[np.arange(batch_size), y_batch])) / batch_size
            loss += 0.5 * reg_param * np.sum(W**)
            dscores = probs
        dscores[np.arange(batch_size), y_batch] -= 1
        dscores /= batch_size

        # Update biases
        db = np.sum(dscores, axis=0)
        b -= learning_rate * db

        # Update weights with regularization
        dW = np.dot(X_batch.T, dscores)
        dW += reg_param * W
        W -= learning_rate * dW

    # Compute training loss and error rate for this epoch
    train_scores = np.dot(X, W) + b
    train_probs = softmax(train_scores, axis=1)
    train_loss = -np.sum(np.log(train_probs[np.arange(num_train), y])) / num_train
    train_loss += 0.5 * reg_param * np.sum(W**2)
    train_losses.append(train_loss)
    train_preds = np.argmax(train_probs, axis=1)
    train_error_rate = np.mean(train_preds != y)
    train_error_rates.append(train_error_rate)

    # Compute validation loss and error rate for this epoch
    val_scores = np.dot(X_val, W) + b
    val_probs = softmax(val_scores, axis=1)
    val_loss = -np.sum(np.log(val_probs[np.arange(num_val), y_val])) / num_val
    val_loss += 0.5 * reg_param * np.sum(W**2)
    val_losses.append(val_loss)
    val_preds = np.argmax(val_probs, axis=1)
    val_error_rate = np.mean(val_preds != y_val)
    val_error_rates.append(val_error_rate)

    # Check if validation loss has stopped decreasing
    if epoch > 0 and val_losses[epoch] > val_losses[epoch-1]:
        break

    return W, b, train_losses, val_losses, train_error_rates, val_error_rates


In [59]:
from scipy.special import softmax, logsumexp

def predict(w,b, X):
    N = len(X)
    predict = []
    for i in range(N):
        if sigmoid(w, X[i], b) >= 0.5: # sigmoid(w,x,b) returns 1/(1+exp(-(dot(x,w)+b)))
            predict.append(1)
        else:
            predict.append(0)
    return np.array(predict)

def sigmoid(w,x,b):
    return 1/(1+math.exp(-(np.dot(x,w)+b)))

def sig_pred(w,X,b):
    a=[]
    for x in X:
        a.append(sigmoid(w,x,b))
#         a.append(1/(1+math.exp(-(np.dot(x,w)+b))))
    return a
    
    
from math import log10
def compute_log_loss(A,B):
    n = len(A)
    res = 0
    for l in zip(A,B):
        res += l[0] * log10(l[1]) + (1 - l[0]) * log10(1 - l[1])                   
    loss = (-1 * res) / n
    return loss

In [60]:
def sgd(X_train,X_test,y_train,y_test,n,w,b):
    del_w=np.zeros_like(X_train[0])
    del_b=0
    wl=[]
    bl=[]
    log_loss_train=[]
    log_loss_test=[]
    train_error_rates=[]
    test_error_rates=[]
    for epoch in range(n):
        for m in range(1000):
            i = np.random.choice(len(X_train))
            del_w=del_w+X_train[i]*(y_train[i]-sigmoid(w,X_train[i],b))
            del_b=del_b+(y_train[i]-sigmoid(w,X_train[i],b))
        w=(1-(alpha/n))*w+alpha*del_w
        b=b+alpha*del_b
        wl.append(w)
        bl.append(b)
        log_loss_train.append(compute_log_loss(y_train,sig_pred(w,X_train,b)))
        log_loss_test.append(compute_log_loss(y_test,sig_pred(w,X_test,b)))
        
        train_error_rates.append(accuracy_score(y_train, pred(w,b, X_train)))
        test_error_rates.append(accuracy_score(y_test, pred(w,b, X_test)))
        
    return log_loss_train,log_loss_test,wl,bl

In [62]:
from sklearn import linear_model
lambdaa = [0,10,100]
lambda_er = []

for ii in lambdaa:
    
    clf = linear_model.SGDClassifier(eta0=0.0001, alpha=ii, loss='log', random_state=15, penalty='l2', tol=1e-3, verbose=2, learning_rate='constant')
    clf.fit(X=mnist_train, y=train_labels)
    label_pred = clf.predict(mnist_dev)
    lambda_er.append(1-accuracy_score(dev_labels, label_pred))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 1.02, NNZs: 708, Bias: -0.156700, T: 42000, Avg. loss: 0.187888
Total training time: 0.10 seconds.
-- Epoch 2
Norm: 1.33, NNZs: 708, Bias: -0.214329, T: 84000, Avg. loss: 0.098149
Total training time: 0.19 seconds.
-- Epoch 3
Norm: 1.53, NNZs: 708, Bias: -0.257526, T: 126000, Avg. loss: 0.079925
Total training time: 0.29 seconds.
-- Epoch 4
Norm: 1.69, NNZs: 708, Bias: -0.292829, T: 168000, Avg. loss: 0.070613
Total training time: 0.40 seconds.
-- Epoch 5
Norm: 1.81, NNZs: 708, Bias: -0.325020, T: 210000, Avg. loss: 0.064682
Total training time: 0.51 seconds.
-- Epoch 6
Norm: 1.91, NNZs: 708, Bias: -0.354095, T: 252000, Avg. loss: 0.060466
Total training time: 0.62 seconds.
-- Epoch 7
Norm: 2.01, NNZs: 708, Bias: -0.379614, T: 294000, Avg. loss: 0.057291
Total training time: 0.71 seconds.
-- Epoch 8
Norm: 2.09, NNZs: 708, Bias: -0.404044, T: 336000, Avg. loss: 0.054764
Total training time: 0.81 seconds.
-- Epoch 9
Norm: 2.16, NNZs: 708, Bias: -0.427389, T: 378000, Avg.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


-- Epoch 1
Norm: 0.96, NNZs: 708, Bias: -0.046455, T: 42000, Avg. loss: 0.169701
Total training time: 0.10 seconds.
-- Epoch 2
Norm: 1.29, NNZs: 708, Bias: -0.049182, T: 84000, Avg. loss: 0.088929
Total training time: 0.19 seconds.
-- Epoch 3
Norm: 1.50, NNZs: 708, Bias: -0.053100, T: 126000, Avg. loss: 0.071302
Total training time: 0.29 seconds.
-- Epoch 4
Norm: 1.65, NNZs: 708, Bias: -0.056162, T: 168000, Avg. loss: 0.062776
Total training time: 0.39 seconds.
-- Epoch 5
Norm: 1.78, NNZs: 708, Bias: -0.061827, T: 210000, Avg. loss: 0.057551
Total training time: 0.48 seconds.
-- Epoch 6
Norm: 1.88, NNZs: 708, Bias: -0.065461, T: 252000, Avg. loss: 0.053962
Total training time: 0.58 seconds.
-- Epoch 7
Norm: 1.97, NNZs: 708, Bias: -0.069017, T: 294000, Avg. loss: 0.051302
Total training time: 0.68 seconds.
-- Epoch 8
Norm: 2.05, NNZs: 708, Bias: -0.073484, T: 336000, Avg. loss: 0.049235
Total training time: 0.77 seconds.
-- Epoch 9
Norm: 2.13, NNZs: 708, Bias: -0.077180, T: 378000, Avg.

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   31.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Norm: 0.08, NNZs: 708, Bias: -1.080408, T: 84000, Avg. loss: 0.353343
Total training time: 0.19 seconds.
-- Epoch 3
Norm: 0.06, NNZs: 708, Bias: -1.347578, T: 126000, Avg. loss: 0.338714
Total training time: 0.28 seconds.
-- Epoch 4
Norm: 0.05, NNZs: 708, Bias: -1.526506, T: 168000, Avg. loss: 0.332824
Total training time: 0.38 seconds.
-- Epoch 5
Norm: 0.05, NNZs: 708, Bias: -1.651029, T: 210000, Avg. loss: 0.330413
Total training time: 0.47 seconds.
-- Epoch 6
Norm: 0.05, NNZs: 708, Bias: -1.738225, T: 252000, Avg. loss: 0.329416
Total training time: 0.57 seconds.
-- Epoch 7
Norm: 0.05, NNZs: 708, Bias: -1.799965, T: 294000, Avg. loss: 0.329142
Total training time: 0.66 seconds.
-- Epoch 8
Norm: 0.05, NNZs: 708, Bias: -1.843928, T: 336000, Avg. loss: 0.329129
Total training time: 0.76 seconds.
-- Epoch 9
Norm: 0.04, NNZs: 708, Bias: -1.875170, T: 378000, Avg. loss: 0.329165
Total training time: 0.85 seconds.
-- Epoch 10
Norm: 0.04, NNZs: 708, Bias: -1.898146, T: 420000, Avg. loss: 0.

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   17.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Norm: 0.01, NNZs: 708, Bias: -1.020191, T: 42000, Avg. loss: 0.489085
Total training time: 0.22 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 708, Bias: -1.494316, T: 84000, Avg. loss: 0.368210
Total training time: 0.45 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 708, Bias: -1.752505, T: 126000, Avg. loss: 0.338307
Total training time: 0.65 seconds.
-- Epoch 4
Norm: 0.00, NNZs: 708, Bias: -1.908045, T: 168000, Avg. loss: 0.328486
Total training time: 0.87 seconds.
-- Epoch 5
Norm: 0.00, NNZs: 708, Bias: -2.007593, T: 210000, Avg. loss: 0.324682
Total training time: 1.09 seconds.
-- Epoch 6
Norm: 0.00, NNZs: 708, Bias: -2.072528, T: 252000, Avg. loss: 0.323167
Total training time: 1.31 seconds.
-- Epoch 7
Norm: 0.00, NNZs: 708, Bias: -2.115139, T: 294000, Avg. loss: 0.322387
Total training time: 1.52 seconds.
-- Epoch 8
Norm: 0.00, NNZs: 708, Bias: -2.144325, T: 336000, Avg. loss: 0.322122
Total training time: 1.73 seconds.
-- Epoch 9
Norm: 0.00, NNZs: 708, Bias: -2.165119, T: 378000, Avg. loss: 0.32

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


-- Epoch 1
Norm: 0.01, NNZs: 708, Bias: -0.979266, T: 42000, Avg. loss: 0.494001
Total training time: 0.10 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 708, Bias: -1.429488, T: 84000, Avg. loss: 0.385495
Total training time: 0.20 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 708, Bias: -1.673025, T: 126000, Avg. loss: 0.359501
Total training time: 0.31 seconds.
-- Epoch 4
Norm: 0.01, NNZs: 708, Bias: -1.814739, T: 168000, Avg. loss: 0.351501
Total training time: 0.41 seconds.
-- Epoch 5
Norm: 0.00, NNZs: 708, Bias: -1.904120, T: 210000, Avg. loss: 0.348648
Total training time: 0.51 seconds.
-- Epoch 6
Norm: 0.00, NNZs: 708, Bias: -1.960575, T: 252000, Avg. loss: 0.347545
Total training time: 0.60 seconds.
-- Epoch 7
Norm: 0.00, NNZs: 708, Bias: -1.997207, T: 294000, Avg. loss: 0.347134
Total training time: 0.70 seconds.
-- Epoch 8
Norm: 0.00, NNZs: 708, Bias: -2.020410, T: 336000, Avg. loss: 0.346944
Total training time: 0.80 seconds.
-- Epoch 9
Norm: 0.00, NNZs: 708, Bias: -2.035488, T: 378000, Avg.

In [ ]:
print(lambda_er)
lambdaa[np.where(lambda_er==np.min(lambda_er))[0][0]]